Author:
Naeimeh Sharghivand <https://www.linkedin.com/in/naeimeh-sharghivand-87666053/>

Last modified: 17-Jun-2023

# Importing required modules

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)

# Importing clean Dataset

In [2]:
# Reading clean datasets
url = "https://drive.google.com/file/d/1usi-wvGB0Gx0H4c5i_wh6zkRiFqeAjh6/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_parquet(path)

url = "https://drive.google.com/file/d/1-6XTcBG6VHRbM7ZYqv02dZ2f_u5v_g7B/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_parquet(path)

url = "https://drive.google.com/file/d/1bLTiAq5G3fLS0r6I-Rggy-o0aKFqZ1-V/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_parquet(path)


# Data quality assessment

In [3]:
products_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9945 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       9945 non-null   object 
 1   name      9945 non-null   object 
 2   desc      9945 non-null   object 
 3   price     9945 non-null   float64
 4   in_stock  9945 non-null   int64  
 5   type      9945 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 543.9+ KB


In [4]:
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   order_id          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


In [5]:
orders_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


1. Exclude unwanted orders

In [6]:
orders_cl['state'].unique()

array(['Cancelled', 'Completed', 'Pending', 'Shopping Basket',
       'Place Order'], dtype=object)

In [7]:
orders_cl['state'].isin(['Cancelled', 'Pending', 'Shopping Basket', 'Place Order']).sum()

180299

In [8]:
# 1. Exclude unwanted orders
mask = orders_cl['state'].isin(['Cancelled', 'Pending', 'Shopping Basket', 'Place Order'])
orders_cl.drop(orders_cl[mask].index[:], inplace=True)

In [9]:
orders_cl['state'].unique()

array(['Completed'], dtype=object)

In [10]:
orders_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46605 entries, 1 to 226619
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      46605 non-null  int64         
 1   created_date  46605 non-null  datetime64[ns]
 2   total_paid    46605 non-null  float64       
 3   state         46605 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.8+ MB


In [11]:
orderlines_cl.rename(columns={'id_order':'order_id'}, inplace=True)

In [12]:
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   order_id          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


2. Make sure that orders in the dataset are in both orders and orderlines (merging orders and orderlines)

In [13]:
# 2. Make sure that orders in the dataset are in both orders and orderlines (merging orders and orderlines)
merged_orders = pd.merge(orders_cl, orderlines_cl, on='order_id', how='inner')
merged_orders = merged_orders.drop('product_id', axis=1)
merged_orders

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date
0,241423,2017-11-06 13:10:02,136.15,Completed,1398738,1,LAC0212,129.16,2017-11-06 12:47:20
1,242832,2017-12-31 17:40:03,15.76,Completed,1529178,1,PAR0074,10.77,2017-12-31 17:26:40
2,243330,2017-02-16 10:59:38,84.98,Completed,1181923,1,OWC0074,77.99,2017-02-15 17:07:44
3,245275,2017-06-28 11:35:37,149.00,Completed,1276706,1,TAD0007,149.00,2017-06-28 11:12:30
4,245595,2017-01-21 12:52:47,112.97,Completed,1154394,2,PAC1561,52.99,2017-01-21 12:49:00
...,...,...,...,...,...,...,...,...,...
56709,527042,2018-03-14 11:47:50,18.98,Completed,1649446,1,APP0927,13.99,2018-03-14 11:42:38
56710,527070,2018-03-14 11:50:48,24.97,Completed,1649512,2,APP0698,9.99,2018-03-14 11:49:01
56711,527074,2018-03-14 11:51:42,24.97,Completed,1649522,2,APP0698,9.99,2018-03-14 11:49:36
56712,527096,2018-03-14 11:58:40,34.96,Completed,1649565,3,APP0698,9.99,2018-03-14 11:54:35


In [14]:
merged_orders.duplicated().any()

False

In [15]:
merged_orders.order_id.duplicated().sum()

13650

In [16]:
merged_orders.id.duplicated().sum()

0

In [17]:
merged_orders.head()

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date
0,241423,2017-11-06 13:10:02,136.15,Completed,1398738,1,LAC0212,129.16,2017-11-06 12:47:20
1,242832,2017-12-31 17:40:03,15.76,Completed,1529178,1,PAR0074,10.77,2017-12-31 17:26:40
2,243330,2017-02-16 10:59:38,84.98,Completed,1181923,1,OWC0074,77.99,2017-02-15 17:07:44
3,245275,2017-06-28 11:35:37,149.00,Completed,1276706,1,TAD0007,149.00,2017-06-28 11:12:30
4,245595,2017-01-21 12:52:47,112.97,Completed,1154394,2,PAC1561,52.99,2017-01-21 12:49:00


3. Exclude orders with unknown products from merged orders and orderlines dataframe



In [18]:
merged_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56714 entries, 0 to 56713
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          56714 non-null  int64         
 1   created_date      56714 non-null  datetime64[ns]
 2   total_paid        56714 non-null  float64       
 3   state             56714 non-null  object        
 4   id                56714 non-null  int64         
 5   product_quantity  56714 non-null  int64         
 6   sku               56714 non-null  object        
 7   unit_price        56714 non-null  float64       
 8   date              56714 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(2)
memory usage: 4.3+ MB


In [19]:
# 3. Exclude orders with unknown products from merged orders and orderlines dataframe
mask = merged_orders['order_id'][~merged_orders['sku'].isin(products_cl['sku'])]
merged_orders.drop(merged_orders[merged_orders['order_id'].isin(mask)].index, inplace=True)

In [20]:
merged_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54306 entries, 0 to 56713
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          54306 non-null  int64         
 1   created_date      54306 non-null  datetime64[ns]
 2   total_paid        54306 non-null  float64       
 3   state             54306 non-null  object        
 4   id                54306 non-null  int64         
 5   product_quantity  54306 non-null  int64         
 6   sku               54306 non-null  object        
 7   unit_price        54306 non-null  float64       
 8   date              54306 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(2)
memory usage: 4.1+ MB


4. Explore the revenue from different tables

In [21]:
# Step 1:
merged_orders['unit_price_total'] = merged_orders['unit_price']*merged_orders['product_quantity']
merged_orders.head()

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date,unit_price_total
0,241423,2017-11-06 13:10:02,136.15,Completed,1398738,1,LAC0212,129.16,2017-11-06 12:47:20,129.16
1,242832,2017-12-31 17:40:03,15.76,Completed,1529178,1,PAR0074,10.77,2017-12-31 17:26:40,10.77
2,243330,2017-02-16 10:59:38,84.98,Completed,1181923,1,OWC0074,77.99,2017-02-15 17:07:44,77.99
3,245275,2017-06-28 11:35:37,149.00,Completed,1276706,1,TAD0007,149.00,2017-06-28 11:12:30,149.00
4,245595,2017-01-21 12:52:47,112.97,Completed,1154394,2,PAC1561,52.99,2017-01-21 12:49:00,105.98


In [22]:
# Step 2: grouped orderlines
#grouped_orderlines = merged_orders.groupby('order_id', as_index=False).agg({'unit_price_total':'sum'})
grouped_orderlines = merged_orders.groupby('order_id', as_index=False)['unit_price_total'].sum()
grouped_orderlines.sample(10)

,order_id,unit_price_total
26743,446461,157.24
37494,504229,12.99
27401,449298,203.00
1304,304758,49.99
2134,308216,12.39
40975,522363,114.99
31301,471090,416.92
15169,387698,258.36
2780,310705,59.99
5784,323234,36.98


In [23]:
grouped_orderlines['unit_price_total'].describe()

count   41688.00
mean      191.01
std       250.81
min         2.49
25%        41.99
50%        94.97
75%       241.79
max      6876.80
Name: unit_price_total, dtype: float64

Step 3: the grouped orderlines table can be merged with the orders table (here it can be merged with the merged_orders) to easily compare the numbers:

In [24]:
# Merging
merged_orders_grouped = pd.merge(merged_orders,grouped_orderlines, on='order_id', how='inner')
merged_orders_grouped.sample(10)

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date,unit_price_total_x,unit_price_total_y
14751,370145,2017-06-28 23:53:57,20.97,Completed,1277293,1,SHE0045,6.99,2017-06-28 23:47:29,6.99,15.98
32975,438982,2017-11-24 13:20:14,329.56,Completed,1436901,1,QNA0150,324.57,2017-11-24 13:05:59,324.57,324.57
53530,522801,2018-03-07 15:30:21,74.98,Completed,1639209,1,WAC0246,69.99,2018-03-07 15:26:20,69.99,69.99
28743,422689,2017-11-06 17:51:09,94.97,Completed,1399067,1,OWC0013,2.99,2017-11-06 17:48:02,2.99,89.98
52207,517355,2018-02-26 00:17:35,252.98,Completed,1626974,1,DLL0037,245.99,2018-02-25 22:39:16,245.99,245.99
45744,489870,2018-01-12 21:49:49,68.98,Completed,1558209,1,SAT0054,64.99,2018-01-12 21:44:54,64.99,64.99
53559,522918,2018-03-07 19:10:18,104.98,Completed,1639447,1,OWC0074,99.99,2018-03-07 19:00:56,99.99,99.99
12506,357961,2017-05-22 13:31:06,345.98,Completed,1252360,1,LAC0205,241.99,2017-05-22 13:26:36,241.99,345.98
35155,446240,2017-11-26 21:33:43,496.72,Completed,1453645,1,APP1435,37.80,2017-11-26 21:23:50,37.80,496.72
8567,326998,2017-02-21 07:41:15,52.97,Completed,1186901,1,KEU0009,25.99,2017-02-21 07:35:03,25.99,48.98


In [25]:
merged_orders.loc[merged_orders['order_id'] == 347309]

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date,unit_price_total
10510,347309,2017-04-19 10:37:26,2221.00,Completed,1230394,3,APP1978,383.00,2017-04-19 10:00:25,1149.00
10511,347309,2017-04-19 10:37:26,2221.00,Completed,1230423,2,APP1982,536.00,2017-04-19 10:32:53,1072.00


In [26]:
merged_orders_grouped.loc[merged_orders_grouped['order_id'] == 347309]

,order_id,created_date,total_paid,state,id,product_quantity,sku,unit_price,date,unit_price_total_x,unit_price_total_y
9819,347309,2017-04-19 10:37:26,2221.00,Completed,1230394,3,APP1978,383.00,2017-04-19 10:00:25,1149.00,2221.00
9820,347309,2017-04-19 10:37:26,2221.00,Completed,1230423,2,APP1982,536.00,2017-04-19 10:32:53,1072.00,2221.00


In [27]:
merged_orders_grouped.rename(columns={'unit_price_total_x': 'unit_price_total', 'unit_price_total_y': 'unit_price_total_sum'}, inplace=True)

In [28]:
merged_orders_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54306 entries, 0 to 54305
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              54306 non-null  int64         
 1   created_date          54306 non-null  datetime64[ns]
 2   total_paid            54306 non-null  float64       
 3   state                 54306 non-null  object        
 4   id                    54306 non-null  int64         
 5   product_quantity      54306 non-null  int64         
 6   sku                   54306 non-null  object        
 7   unit_price            54306 non-null  float64       
 8   date                  54306 non-null  datetime64[ns]
 9   unit_price_total      54306 non-null  float64       
 10  unit_price_total_sum  54306 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(3), object(2)
memory usage: 5.0+ MB


In [29]:
merged_orders_grouped.describe()

,order_id,total_paid,id,product_quantity,unit_price,unit_price_total,unit_price_total_sum
count,54306.00,54306.00,54306.00,54306.00,54306.00,54306.00,54306.00
mean,414377.40,222.52,1385530.93,1.13,135.33,146.63,218.14
std,65546.96,283.32,150938.49,0.82,180.56,208.80,283.05
min,241423.00,0.00,1119116.00,1.00,1.00,1.99,2.49
25%,362751.50,53.98,1264893.00,1.00,26.99,29.69,49.99
50%,417123.00,114.25,1389349.50,1.00,64.99,69.29,109.98
75%,469853.50,288.99,1513205.75,1.00,154.00,166.00,284.79
max,527112.00,6876.94,1649593.00,72.00,999.99,6876.80,6876.80


In [32]:
from google.colab import files

merged_orders_grouped.to_csv('merged_orders.csv', index=False)
files.download('merged_orders.csv')

merged_orders_grouped.to_parquet('merged_orders.parquet')
files.download('merged_orders.parquet')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>